In [1]:
# # Filter top 15 assets by Sharpe ratio before QAOA optimization
# import numpy as np
# import pandas as pd

# # Load expected returns and covariance matrix
# expected_returns = pd.read_csv("results/expected_returns.csv", index_col=0)
# cov_matrix = pd.read_csv("results/cov_matrix.csv", index_col=0)

# # Calculate Sharpe ratio for each asset (assuming risk-free rate = 0)
# asset_volatility = np.sqrt(np.diag(cov_matrix))
# sharpe_ratio = expected_returns.values.flatten() / asset_volatility

# # Get indices of top 15 assets by Sharpe ratio
# N = 15
# sorted_indices = np.argsort(sharpe_ratio)[::-1][:N]
# top_assets = expected_returns.index[sorted_indices]

# # Filter expected returns and covariance matrix for top assets
# mu = expected_returns.loc[top_assets].values.flatten()
# Sigma = cov_matrix.loc[top_assets, top_assets].values
# n_assets = N

In [2]:
import numpy as np
import pandas as pd
from qiskit_optimization import QuadraticProgram

In [3]:
expected_returns = pd.read_csv("results/expected_returns.csv", index_col=0)
cov_matrix = pd.read_csv("results/cov_matrix.csv", index_col=0)

In [4]:
# Calculate Sharpe ratio for each asset (assuming risk-free rate = 0)
asset_volatility = np.sqrt(np.diag(cov_matrix))
sharpe_ratio = expected_returns.values.flatten() / asset_volatility

In [5]:
# Get indices of top 15 assets by Sharpe ratio
N = 15
sorted_indices = np.argsort(sharpe_ratio)[::-1][:N]
top_assets = expected_returns.index[sorted_indices]

In [6]:

# Filter expected returns and covariance matrix for top assets
mu = expected_returns.loc[top_assets].values.flatten()
Sigma = cov_matrix.loc[top_assets, top_assets].values
n_assets = N

In [7]:
gamma = 0.5

# Flexible budget: you can choose either by number or percentage
# Example: set budget_type = "percentage" and budget_value = 0.3 (30%)
#          or set budget_type = "number" and budget_value = 10 (10 stocks)

budget_type = "number"   # options: "percentage" or "number"
budget_value = 5         # if percentage → fraction of total assets, if number → count

if budget_type == "percentage":
    budget = int(n_assets * budget_value)
else:  # "number"
    budget = int(budget_value)

# Ensure budget is within valid range
budget = max(1, min(budget, n_assets))
print(f"Budget constraint: selecting {budget} out of {n_assets} assets")

Budget constraint: selecting 5 out of 15 assets


In [8]:
qp = QuadraticProgram("Portfolio Optimization")

In [9]:
# Decision variables: x_i = 1 if asset i is chosen, 0 otherwise
for i in range(n_assets):
    qp.binary_var(name=f"x_{i}")

In [10]:
# Objective: maximize returns - gamma * risk
# Equivalent to: minimize -mu^T x + gamma * x^T Σ x
linear = -mu
quadratic = gamma * Sigma
qp.minimize(linear=linear, quadratic=quadratic)

In [11]:
# Budget constraint: select exactly "budget" assets
qp.linear_constraint(
    linear={f"x_{i}": 1 for i in range(n_assets)},
    sense="==",
    rhs=budget,
    name="budget_constraint"
)

<LinearConstraint: x_0 + x_1 + x_10 + x_11 + x_12 + x_13 + x_14 + x_2... == 5 'budget_constraint'>

In [12]:
import numpy as np
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [13]:
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=1)
optimizer = MinimumEigenOptimizer(qaoa)

In [14]:
result = optimizer.solve(qp)

In [15]:
print("\nOptimal portfolio allocation:")
print(result)


Optimal portfolio allocation:
fval=-0.007587618043762753, x_0=1.0, x_1=1.0, x_2=0.0, x_3=1.0, x_4=1.0, x_5=0.0, x_6=1.0, x_7=0.0, x_8=0.0, x_9=0.0, x_10=0.0, x_11=0.0, x_12=0.0, x_13=0.0, x_14=0.0, status=SUCCESS


In [16]:
selected_assets = [expected_returns.index[i] for i, x in enumerate(result.x) if x > 0.5]
print("\nSelected Portfolio Assets:", selected_assets)


Selected Portfolio Assets: ['Adani Enterprises', 'Adani Green Energy', 'Apollo Hospitals', 'Asian Paints', 'Bajaj Auto']
